# Model Evaluation for Cryptocurrency Volatility Prediction

This notebook focuses on comprehensive evaluation of trained models, including:
- Performance metrics analysis
- Cross-validation results
- Feature importance analysis
- Model comparison
- Prediction analysis
- Business metrics evaluation

## Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
import joblib

# Statistical libraries
from scipy import stats
from scipy.stats import pearsonr, spearmanr

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8')

print("Model Evaluation Notebook Initialized!")

In [ ]:
# Load data and models
try:
    # Load test data
    X_test = pd.read_csv('../data/X_test.csv', index_col=0)
    y_test = pd.read_csv('../data/y_test.csv', index_col=0)['target']
    
    # Load trained models
    rf_model = joblib.load('../models/random_forest_model.pkl')
    gb_model = joblib.load('../models/gradient_boosting_model.pkl')
    ensemble_model = joblib.load('../models/ensemble_model.pkl')
    
    print(f"Test data loaded: X_test {X_test.shape}, y_test {y_test.shape}")
    print(f"Models loaded successfully!")
    
except FileNotFoundError as e:
    print(f"Could not load required files: {e}")
    print("Please ensure model training has been completed first.")
    
    # Create dummy data for demonstration
    print("Creating dummy data for demonstration...")
    np.random.seed(42)
    X_test = pd.DataFrame(np.random.randn(1000, 50), 
                         columns=[f'feature_{i}' for i in range(50)])
    y_test = pd.Series(np.random.rand(1000) * 0.1, name='target')
    
    # Create dummy models
    rf_model = RandomForestRegressor(n_estimators=10, random_state=42)
    gb_model = GradientBoostingRegressor(n_estimators=10, random_state=42)
    
    # Quick training
    rf_model.fit(X_test[:800], y_test[:800])
    gb_model.fit(X_test[:800], y_test[:800])
    
    # Use test set for evaluation
    X_test = X_test[800:]
    y_test = y_test[800:]

## 1. Model Predictions

In [ ]:
# Generate predictions from all models
print("=== GENERATING MODEL PREDICTIONS ===\n")

# Individual model predictions
rf_predictions = rf_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)

# Ensemble prediction (weighted average)
if 'ensemble_model' in locals():
    ensemble_predictions = ensemble_model.predict(X_test)
else:
    # Simple ensemble as weighted average
    ensemble_predictions = 0.6 * gb_predictions + 0.4 * rf_predictions

# Store predictions in DataFrame
predictions_df = pd.DataFrame({
    'actual': y_test.values,
    'rf_pred': rf_predictions,
    'gb_pred': gb_predictions,
    'ensemble_pred': ensemble_predictions
}, index=y_test.index)

print(f"Predictions generated for {len(predictions_df)} samples")
print(f"\nPrediction Statistics:")
print(predictions_df.describe())

# Display sample predictions
print(f"\nSample Predictions:")
print(predictions_df.head(10))

## 2. Performance Metrics

In [ ]:
# Calculate comprehensive performance metrics
def calculate_metrics(y_true, y_pred, model_name):
    """Calculate comprehensive performance metrics."""
    
    # Basic metrics
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    
    # Additional metrics
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    # Directional accuracy
    y_true_diff = np.diff(y_true)
    y_pred_diff = np.diff(y_pred)
    directional_accuracy = np.mean(np.sign(y_true_diff) == np.sign(y_pred_diff)) * 100
    
    # Correlation metrics
    pearson_corr, _ = pearsonr(y_true, y_pred)
    spearman_corr, _ = spearmanr(y_true, y_pred)
    
    # Residual analysis
    residuals = y_true - y_pred
    residual_std = np.std(residuals)
    residual_skew = stats.skew(residuals)
    residual_kurtosis = stats.kurtosis(residuals)
    
    return {
        'Model': model_name,
        'R²': r2,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'Directional Accuracy': directional_accuracy,
        'Pearson Correlation': pearson_corr,
        'Spearman Correlation': spearman_corr,
        'Residual Std': residual_std,
        'Residual Skewness': residual_skew,
        'Residual Kurtosis': residual_kurtosis
    }

# Calculate metrics for all models
print("=== PERFORMANCE METRICS ===\n")

metrics_list = []
metrics_list.append(calculate_metrics(y_test, rf_predictions, 'Random Forest'))
metrics_list.append(calculate_metrics(y_test, gb_predictions, 'Gradient Boosting'))
metrics_list.append(calculate_metrics(y_test, ensemble_predictions, 'Ensemble'))

metrics_df = pd.DataFrame(metrics_list)
print(metrics_df.round(4))

# Best model identification
best_model_r2 = metrics_df.loc[metrics_df['R²'].idxmax(), 'Model']
best_model_rmse = metrics_df.loc[metrics_df['RMSE'].idxmin(), 'Model']
best_model_mae = metrics_df.loc[metrics_df['MAE'].idxmin(), 'Model']

print(f"\n🏆 Best Models by Metric:")
print(f"   • Highest R²: {best_model_r2} ({metrics_df['R²'].max():.4f})")
print(f"   • Lowest RMSE: {best_model_rmse} ({metrics_df['RMSE'].min():.4f})")
print(f"   • Lowest MAE: {best_model_mae} ({metrics_df['MAE'].min():.4f})")

## 3. Visualization of Results

In [ ]:
# Comprehensive visualization of model performance
print("=== CREATING PERFORMANCE VISUALIZATIONS ===\n")

# 1. Predicted vs Actual scatter plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Predicted vs Actual Volatility', fontsize=16)

models = [('Random Forest', rf_predictions), ('Gradient Boosting', gb_predictions), 
          ('Ensemble', ensemble_predictions)]

for idx, (name, preds) in enumerate(models):
    row, col = idx // 2, idx % 2
    ax = axes[row, col]
    
    # Scatter plot
    ax.scatter(y_test, preds, alpha=0.6, s=20)
    
    # Perfect prediction line
    min_val = min(y_test.min(), preds.min())
    max_val = max(y_test.max(), preds.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    
    # Add R² score
    r2 = r2_score(y_test, preds)
    ax.text(0.05, 0.95, f'R² = {r2:.4f}', transform=ax.transAxes, 
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.set_xlabel('Actual Volatility')
    ax.set_ylabel('Predicted Volatility')
    ax.set_title(f'{name}')
    ax.grid(True, alpha=0.3)

# Remove empty subplot
axes[1, 1].remove()

plt.tight_layout()
plt.show()

# 2. Residual analysis
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Residual Analysis', fontsize=16)

for idx, (name, preds) in enumerate(models):
    residuals = y_test - preds
    
    # Residuals vs Predicted
    axes[0, idx].scatter(preds, residuals, alpha=0.6, s=20)
    axes[0, idx].axhline(y=0, color='r', linestyle='--')
    axes[0, idx].set_xlabel('Predicted Values')
    axes[0, idx].set_ylabel('Residuals')
    axes[0, idx].set_title(f'{name} - Residuals vs Predicted')
    axes[0, idx].grid(True, alpha=0.3)
    
    # Residual distribution
    axes[1, idx].hist(residuals, bins=30, alpha=0.7, density=True)
    axes[1, idx].axvline(residuals.mean(), color='r', linestyle='--', 
                        label=f'Mean: {residuals.mean():.4f}')
    axes[1, idx].set_xlabel('Residuals')
    axes[1, idx].set_ylabel('Density')
    axes[1, idx].set_title(f'{name} - Residual Distribution')
    axes[1, idx].legend()
    axes[1, idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 3. Model comparison metrics
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# R² comparison
r2_scores = [r2_score(y_test, preds) for _, preds in models]
axes[0].bar([name for name, _ in models], r2_scores, color=['blue', 'green', 'red'])
axes[0].set_ylabel('R² Score')
axes[0].set_title('R² Score Comparison')
axes[0].grid(True, alpha=0.3)

# RMSE comparison
rmse_scores = [np.sqrt(mean_squared_error(y_test, preds)) for _, preds in models]
axes[1].bar([name for name, _ in models], rmse_scores, color=['blue', 'green', 'red'])
axes[1].set_ylabel('RMSE')
axes[1].set_title('RMSE Comparison')
axes[1].grid(True, alpha=0.3)

# MAE comparison
mae_scores = [mean_absolute_error(y_test, preds) for _, preds in models]
axes[2].bar([name for name, _ in models], mae_scores, color=['blue', 'green', 'red'])
axes[2].set_ylabel('MAE')
axes[2].set_title('MAE Comparison')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Performance visualizations completed!")

## 4. Feature Importance Analysis

In [ ]:
# Feature importance analysis
print("=== FEATURE IMPORTANCE ANALYSIS ===\n")

# Extract feature importance from tree-based models
rf_importance = pd.DataFrame({
    'feature': X_test.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

gb_importance = pd.DataFrame({
    'feature': X_test.columns,
    'importance': gb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 15 Features - Random Forest:")
print(rf_importance.head(15))

print("\nTop 15 Features - Gradient Boosting:")
print(gb_importance.head(15))

# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Random Forest importance
top_rf = rf_importance.head(15)
axes[0].barh(range(len(top_rf)), top_rf['importance'])
axes[0].set_yticks(range(len(top_rf)))
axes[0].set_yticklabels(top_rf['feature'])
axes[0].set_xlabel('Importance')
axes[0].set_title('Random Forest - Top 15 Feature Importance')
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3)

# Gradient Boosting importance
top_gb = gb_importance.head(15)
axes[1].barh(range(len(top_gb)), top_gb['importance'])
axes[1].set_yticks(range(len(top_gb)))
axes[1].set_yticklabels(top_gb['feature'])
axes[1].set_xlabel('Importance')
axes[1].set_title('Gradient Boosting - Top 15 Feature Importance')
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Combined importance (average)
combined_importance = pd.merge(rf_importance, gb_importance, on='feature', suffixes=('_rf', '_gb'))
combined_importance['avg_importance'] = (combined_importance['importance_rf'] + 
                                       combined_importance['importance_gb']) / 2
combined_importance = combined_importance.sort_values('avg_importance', ascending=False)

print("\nTop 15 Features - Combined Importance:")
print(combined_importance[['feature', 'avg_importance']].head(15))

# Save feature importance
combined_importance.to_csv('../data/model_feature_importance.csv', index=False)
print("\nFeature importance saved to ../data/model_feature_importance.csv")

## 5. Time Series Analysis

In [ ]:
# Time series prediction analysis
print("=== TIME SERIES PREDICTION ANALYSIS ===\n")

# Create time series plots
if hasattr(y_test.index, 'to_datetime'):
    time_index = pd.to_datetime(y_test.index)
else:
    time_index = range(len(y_test))

# Plot actual vs predicted over time
fig, axes = plt.subplots(3, 1, figsize=(15, 12))
fig.suptitle('Time Series Predictions vs Actual', fontsize=16)

models_ts = [('Random Forest', rf_predictions), ('Gradient Boosting', gb_predictions), 
             ('Ensemble', ensemble_predictions)]

for idx, (name, preds) in enumerate(models_ts):
    # Sample data for visualization (plot every 10th point if too many)
    step = max(1, len(y_test) // 200)
    
    axes[idx].plot(time_index[::step], y_test.iloc[::step], 'b-', alpha=0.7, label='Actual', linewidth=1)
    axes[idx].plot(time_index[::step], preds[::step], 'r-', alpha=0.7, label='Predicted', linewidth=1)
    axes[idx].set_ylabel('Volatility')
    axes[idx].set_title(f'{name}')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

axes[-1].set_xlabel('Time')
plt.tight_layout()
plt.show()

# Prediction error over time
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Prediction Errors Over Time', fontsize=16)

for idx, (name, preds) in enumerate(models_ts):
    row, col = idx // 2, idx % 2
    if idx < 3:
        errors = y_test - preds
        axes[row, col].plot(time_index[::step], errors[::step], alpha=0.7)
        axes[row, col].axhline(y=0, color='r', linestyle='--')
        axes[row, col].set_ylabel('Prediction Error')
        axes[row, col].set_title(f'{name} - Prediction Errors')
        axes[row, col].grid(True, alpha=0.3)

# Remove empty subplot
if len(models_ts) < 4:
    axes[1, 1].remove()

plt.tight_layout()
plt.show()

# Rolling performance metrics
window_size = max(20, len(y_test) // 20)
rolling_r2 = []
rolling_rmse = []

for i in range(window_size, len(y_test)):
    window_actual = y_test.iloc[i-window_size:i]
    window_pred = ensemble_predictions[i-window_size:i]
    
    r2 = r2_score(window_actual, window_pred)
    rmse = np.sqrt(mean_squared_error(window_actual, window_pred))
    
    rolling_r2.append(r2)
    rolling_rmse.append(rmse)

# Plot rolling metrics
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(rolling_r2)
axes[0].set_title(f'Rolling R² Score (window={window_size})')
axes[0].set_ylabel('R² Score')
axes[0].grid(True, alpha=0.3)

axes[1].plot(rolling_rmse)
axes[1].set_title(f'Rolling RMSE (window={window_size})')
axes[1].set_ylabel('RMSE')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Rolling performance analysis completed (window size: {window_size})")

## 6. Volatility Classification Analysis

In [ ]:
# Volatility classification analysis
print("=== VOLATILITY CLASSIFICATION ANALYSIS ===\n")

# Define volatility regimes
def classify_volatility(vol_values):
    """Classify volatility into regimes."""
    low_threshold = np.percentile(vol_values, 33)
    high_threshold = np.percentile(vol_values, 67)
    
    conditions = [
        vol_values <= low_threshold,
        (vol_values > low_threshold) & (vol_values <= high_threshold),
        vol_values > high_threshold
    ]
    choices = ['Low', 'Medium', 'High']
    
    return np.select(conditions, choices)

# Classify actual and predicted volatilities
actual_classes = classify_volatility(y_test)
ensemble_classes = classify_volatility(ensemble_predictions)

# Classification accuracy
class_accuracy = (actual_classes == ensemble_classes).mean()
print(f"Volatility regime classification accuracy: {class_accuracy:.3f}")

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(actual_classes, ensemble_classes, labels=['Low', 'Medium', 'High'])
print(f"\nConfusion Matrix:")
print(pd.DataFrame(cm, index=['Low', 'Medium', 'High'], columns=['Low', 'Medium', 'High']))

print(f"\nClassification Report:")
print(classification_report(actual_classes, ensemble_classes))

# Visualize classification performance
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Confusion matrix heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Low', 'Medium', 'High'],
            yticklabels=['Low', 'Medium', 'High'], ax=axes[0])
axes[0].set_title('Volatility Classification Confusion Matrix')
axes[0].set_ylabel('Actual')
axes[0].set_xlabel('Predicted')

# Class distribution
actual_counts = pd.Series(actual_classes).value_counts()
pred_counts = pd.Series(ensemble_classes).value_counts()

x = np.arange(len(actual_counts))
width = 0.35

axes[1].bar(x - width/2, actual_counts.values, width, label='Actual', alpha=0.7)
axes[1].bar(x + width/2, pred_counts.values, width, label='Predicted', alpha=0.7)
axes[1].set_xlabel('Volatility Regime')
axes[1].set_ylabel('Count')
axes[1].set_title('Volatility Regime Distribution')
axes[1].set_xticks(x)
axes[1].set_xticklabels(actual_counts.index)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Performance by volatility regime
print(f"\nPerformance by Volatility Regime:")
for regime in ['Low', 'Medium', 'High']:
    regime_mask = actual_classes == regime
    if regime_mask.sum() > 0:
        regime_r2 = r2_score(y_test[regime_mask], ensemble_predictions[regime_mask])
        regime_rmse = np.sqrt(mean_squared_error(y_test[regime_mask], ensemble_predictions[regime_mask]))
        print(f"  {regime} Volatility: R² = {regime_r2:.4f}, RMSE = {regime_rmse:.4f} (n={regime_mask.sum()})")

## 7. Business Impact Analysis

In [ ]:
# Business impact analysis
print("=== BUSINESS IMPACT ANALYSIS ===\n")

# Trading strategy simulation
def simulate_trading_strategy(actual_vol, predicted_vol, threshold=0.02):
    """Simulate a volatility-based trading strategy."""
    
    # Strategy: Buy when predicted volatility is low, sell when high
    signals = np.where(predicted_vol < threshold, 1,  # Buy signal
                      np.where(predicted_vol > threshold * 2, -1, 0))  # Sell signal
    
    # Simulate returns (assuming inverse relationship with volatility)
    # Higher volatility = higher risk = potential for higher returns but more losses
    base_returns = np.random.normal(0.001, actual_vol)  # Daily returns based on actual volatility
    
    # Strategy returns
    strategy_returns = signals * base_returns
    
    # Calculate cumulative returns
    cumulative_returns = (1 + strategy_returns).cumprod()
    
    return {
        'total_return': cumulative_returns[-1] - 1,
        'sharpe_ratio': strategy_returns.mean() / strategy_returns.std() * np.sqrt(252),
        'max_drawdown': (cumulative_returns / cumulative_returns.expanding().max() - 1).min(),
        'win_rate': (strategy_returns > 0).mean(),
        'num_trades': (signals != 0).sum()
    }

# Simulate strategies
perfect_strategy = simulate_trading_strategy(y_test, y_test)  # Perfect predictions
ensemble_strategy = simulate_trading_strategy(y_test, ensemble_predictions)  # Our model
random_strategy = simulate_trading_strategy(y_test, np.random.rand(len(y_test)) * y_test.std())  # Random

# Results comparison
strategy_results = pd.DataFrame({
    'Perfect Predictions': perfect_strategy,
    'Ensemble Model': ensemble_strategy,
    'Random Predictions': random_strategy
})

print("Trading Strategy Simulation Results:")
print(strategy_results.round(4))

# Risk assessment accuracy
def assess_risk_prediction_accuracy(actual_vol, predicted_vol, risk_threshold=0.03):
    """Assess accuracy of risk predictions."""
    
    actual_high_risk = actual_vol > risk_threshold
    predicted_high_risk = predicted_vol > risk_threshold
    
    # Calculate risk prediction metrics
    tp = ((actual_high_risk) & (predicted_high_risk)).sum()  # True positives
    tn = ((~actual_high_risk) & (~predicted_high_risk)).sum()  # True negatives
    fp = ((~actual_high_risk) & (predicted_high_risk)).sum()  # False positives
    fn = ((actual_high_risk) & (~predicted_high_risk)).sum()  # False negatives
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'true_positives': tp,
        'false_positives': fp,
        'false_negatives': fn
    }

risk_assessment = assess_risk_prediction_accuracy(y_test, ensemble_predictions)

print(f"\nRisk Prediction Assessment:")
for metric, value in risk_assessment.items():
    if isinstance(value, float):
        print(f"  {metric.replace('_', ' ').title()}: {value:.4f}")
    else:
        print(f"  {metric.replace('_', ' ').title()}: {value}")

# Volatility forecasting accuracy by horizon
print(f"\nVolatility Forecasting Accuracy:")
print(f"  Mean Absolute Error: {mean_absolute_error(y_test, ensemble_predictions):.6f}")
print(f"  Mean Absolute Percentage Error: {np.mean(np.abs((y_test - ensemble_predictions) / y_test)) * 100:.2f}%")
print(f"  Correlation with Actual: {np.corrcoef(y_test, ensemble_predictions)[0,1]:.4f}")

# Economic impact estimation
portfolio_value = 1000000  # $1M portfolio
risk_reduction = abs(ensemble_strategy['max_drawdown'] - random_strategy['max_drawdown'])
potential_savings = portfolio_value * risk_reduction

print(f"\nEstimated Economic Impact:")
print(f"  Portfolio Value: ${portfolio_value:,.0f}")
print(f"  Max Drawdown Reduction: {risk_reduction:.2%}")
print(f"  Potential Risk Savings: ${potential_savings:,.0f}")
print(f"  Sharpe Ratio Improvement: {ensemble_strategy['sharpe_ratio'] - random_strategy['sharpe_ratio']:.3f}")

## 8. Model Robustness Analysis

In [ ]:
# Model robustness and stability analysis
print("=== MODEL ROBUSTNESS ANALYSIS ===\n")

# Bootstrap confidence intervals
def bootstrap_metric(y_true, y_pred, metric_func, n_bootstrap=1000):
    """Calculate bootstrap confidence intervals for metrics."""
    bootstrap_scores = []
    n_samples = len(y_true)
    
    for _ in range(n_bootstrap):
        # Bootstrap sample
        indices = np.random.choice(n_samples, n_samples, replace=True)
        y_true_boot = y_true.iloc[indices] if hasattr(y_true, 'iloc') else y_true[indices]
        y_pred_boot = y_pred[indices]
        
        # Calculate metric
        score = metric_func(y_true_boot, y_pred_boot)
        bootstrap_scores.append(score)
    
    return np.array(bootstrap_scores)

# Calculate confidence intervals for key metrics
print("Bootstrap Confidence Intervals (95%):")

# R² confidence interval
r2_bootstrap = bootstrap_metric(y_test, ensemble_predictions, r2_score)
r2_ci = np.percentile(r2_bootstrap, [2.5, 97.5])
print(f"  R² Score: {r2_score(y_test, ensemble_predictions):.4f} [{r2_ci[0]:.4f}, {r2_ci[1]:.4f}]")

# RMSE confidence interval
rmse_bootstrap = bootstrap_metric(y_test, ensemble_predictions, 
                                 lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)))
rmse_ci = np.percentile(rmse_bootstrap, [2.5, 97.5])
print(f"  RMSE: {np.sqrt(mean_squared_error(y_test, ensemble_predictions)):.4f} [{rmse_ci[0]:.4f}, {rmse_ci[1]:.4f}]")

# MAE confidence interval
mae_bootstrap = bootstrap_metric(y_test, ensemble_predictions, mean_absolute_error)
mae_ci = np.percentile(mae_bootstrap, [2.5, 97.5])
print(f"  MAE: {mean_absolute_error(y_test, ensemble_predictions):.4f} [{mae_ci[0]:.4f}, {mae_ci[1]:.4f}]")

# Prediction stability analysis
# Add small noise to features and see how predictions change
noise_levels = [0.01, 0.05, 0.1, 0.2]
stability_results = []

original_predictions = ensemble_predictions.copy()

for noise_level in noise_levels:
    # Add noise to features
    X_noisy = X_test + np.random.normal(0, noise_level, X_test.shape)
    
    # Generate predictions with noisy features
    if hasattr(gb_model, 'predict'):
        noisy_predictions = gb_model.predict(X_noisy)
    else:
        noisy_predictions = original_predictions + np.random.normal(0, noise_level * 0.1, len(original_predictions))
    
    # Calculate stability metric (correlation with original predictions)
    stability = np.corrcoef(original_predictions, noisy_predictions)[0, 1]
    stability_results.append(stability)

print(f"\nPrediction Stability Analysis:")
for noise_level, stability in zip(noise_levels, stability_results):
    print(f"  Noise Level {noise_level}: Correlation = {stability:.4f}")

# Visualize stability
plt.figure(figsize=(10, 6))
plt.plot(noise_levels, stability_results, 'o-', linewidth=2, markersize=8)
plt.xlabel('Noise Level')
plt.ylabel('Prediction Stability (Correlation)')
plt.title('Model Prediction Stability vs Input Noise')
plt.grid(True, alpha=0.3)
plt.ylim(0, 1)
plt.show()

# Outlier impact analysis
# Remove extreme outliers and see how performance changes
outlier_threshold = np.percentile(y_test, 95)
non_outlier_mask = y_test <= outlier_threshold

print(f"\nOutlier Impact Analysis:")
print(f"  Original dataset size: {len(y_test)}")
print(f"  Non-outlier subset size: {non_outlier_mask.sum()}")

# Performance on non-outlier subset
non_outlier_r2 = r2_score(y_test[non_outlier_mask], ensemble_predictions[non_outlier_mask])
non_outlier_rmse = np.sqrt(mean_squared_error(y_test[non_outlier_mask], ensemble_predictions[non_outlier_mask]))

print(f"  Performance without outliers:")
print(f"    R² Score: {non_outlier_r2:.4f} (vs {r2_score(y_test, ensemble_predictions):.4f} with outliers)")
print(f"    RMSE: {non_outlier_rmse:.4f} (vs {np.sqrt(mean_squared_error(y_test, ensemble_predictions)):.4f} with outliers)")

print(f"\nModel robustness analysis completed!")

## Summary and Conclusions

In [ ]:
# Comprehensive evaluation summary
print("=== MODEL EVALUATION SUMMARY ===\n")

# Best model identification
best_model = metrics_df.loc[metrics_df['R²'].idxmax()]

print(f"🏆 BEST PERFORMING MODEL: {best_model['Model']}")
print(f"   • R² Score: {best_model['R²']:.4f}")
print(f"   • RMSE: {best_model['RMSE']:.4f}")
print(f"   • MAE: {best_model['MAE']:.4f}")
print(f"   • Directional Accuracy: {best_model['Directional Accuracy']:.1f}%")

print(f"\n📊 MODEL COMPARISON:")
for _, row in metrics_df.iterrows():
    print(f"   • {row['Model']}: R² = {row['R²']:.4f}, RMSE = {row['RMSE']:.4f}")

print(f"\n🎯 KEY PERFORMANCE INSIGHTS:")
print(f"   • Ensemble method shows {'best' if best_model['Model'] == 'Ensemble' else 'competitive'} performance")
print(f"   • Volatility classification accuracy: {class_accuracy:.1%}")
print(f"   • Risk prediction precision: {risk_assessment['precision']:.3f}")
print(f"   • Model predictions are {'stable' if min(stability_results) > 0.8 else 'moderately stable'} under noise")

print(f"\n💼 BUSINESS VALUE:")
print(f"   • Trading strategy Sharpe ratio: {ensemble_strategy['sharpe_ratio']:.3f}")
print(f"   • Maximum drawdown: {ensemble_strategy['max_drawdown']:.2%}")
print(f"   • Risk reduction potential: ${potential_savings:,.0f} on $1M portfolio")
print(f"   • High-risk event detection recall: {risk_assessment['recall']:.3f}")

print(f"\n🔍 TOP PREDICTIVE FEATURES:")
top_5_features = combined_importance.head(5)
for idx, row in top_5_features.iterrows():
    print(f"   {idx+1}. {row['feature']}: {row['avg_importance']:.4f}")

print(f"\n⚠️ MODEL LIMITATIONS:")
print(f"   • Performance varies by volatility regime")
print(f"   • Outliers impact prediction accuracy")
print(f"   • Feature noise sensitivity: {1 - min(stability_results):.1%} correlation drop at 20% noise")
print(f"   • Temporal stability needs monitoring in deployment")

print(f"\n✅ MODEL READINESS ASSESSMENT:")
readiness_score = 0
criteria = {
    'R² Score > 0.7': best_model['R²'] > 0.7,
    'Classification Accuracy > 70%': class_accuracy > 0.7,
    'Risk Precision > 0.6': risk_assessment['precision'] > 0.6,
    'Stability > 0.8': min(stability_results) > 0.8,
    'Business Value Positive': ensemble_strategy['sharpe_ratio'] > 0
}

for criterion, passed in criteria.items():
    status = "✅" if passed else "❌"
    print(f"   {status} {criterion}")
    if passed:
        readiness_score += 1

readiness_percentage = (readiness_score / len(criteria)) * 100
print(f"\n📈 OVERALL READINESS SCORE: {readiness_score}/{len(criteria)} ({readiness_percentage:.0f}%)")

if readiness_percentage >= 80:
    recommendation = "✅ RECOMMENDED FOR PRODUCTION DEPLOYMENT"
elif readiness_percentage >= 60:
    recommendation = "⚠️ REQUIRES IMPROVEMENTS BEFORE DEPLOYMENT"
else:
    recommendation = "❌ NOT READY FOR PRODUCTION"

print(f"\n🎯 DEPLOYMENT RECOMMENDATION: {recommendation}")

print(f"\n📁 EVALUATION OUTPUTS:")
print(f"   • Model performance metrics calculated")
print(f"   • Feature importance rankings saved")
print(f"   • Prediction results analyzed")
print(f"   • Business impact assessment completed")
print(f"   • Robustness analysis performed")

# Save evaluation results
evaluation_summary = {
    'best_model': best_model['Model'],
    'performance_metrics': metrics_df.to_dict('records'),
    'classification_accuracy': class_accuracy,
    'risk_assessment': risk_assessment,
    'business_metrics': ensemble_strategy,
    'stability_analysis': dict(zip(noise_levels, stability_results)),
    'readiness_score': readiness_percentage
}

import json
with open('../data/evaluation_summary.json', 'w') as f:
    json.dump(evaluation_summary, f, indent=2, default=str)

print(f"\n💾 Evaluation summary saved to ../data/evaluation_summary.json")

print(f"\n" + "="*60)
print("🎉 MODEL EVALUATION COMPLETED SUCCESSFULLY!")
print("   Ready for deployment decision and monitoring setup")
print("="*60)